<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_graphic_icecore_back_2K_ce_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sio-co2o2/keelingcurve_notebooks/main?labpath=notebooks/create_graphic_icecore_back_2K_ce_keelingcurve.ipynb)

# Create a graphic appearing on the Keeling Curve website
### **Graphic: CO<sub>2</sub> record from 2K years ago to now**

**Mauna Loa, Hawaii CO<sub>2</sub> record starting in 1958 and ice-core CO<sub>2</sub> record before 1958**

#### This notebook creates customizable PDF and PNG images of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

For an overview of the Keeling Curve notebooks, data and graphics licenses, and how to use the notebooks, see [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb).

**To use the notebook**, first run all the cells. In Google Colab, select from the top menu and select Runtime -> Run all. Then the buttons will appear to click on which creates and downloads the graphics.

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

---

### [Go to creating the graphic section](#button-to-create-and-save-the-graphic)

---

### Sample of the graphic produced by this notebook

![title](https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_graphics/co2_2k_ce.png?raw=1)

## **Notebook Code**

### Import python packages

In [1]:
import functools
import requests
import re
import ssl
import pandas as pd
import numpy as np
from scipy import interpolate
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker
from matplotlib import rc,rcParams
from matplotlib.path import Path
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.transforms
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import urllib
import os

%matplotlib inline

In [2]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [3]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

### Set directories and file names

In [4]:
# Get MLO data from the keelingcurve_notebooks github repository
mlo_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_full_record_now_span.csv'

# Get the icecore dataset going back 2K years
# This is a subset of the full dataset found at https://doi.org/10.25919/5bfe29ff807fb
icecore_2K_data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/icecore_data/Law_Dome_GHG_2000years_co2_by_age.csv'

# Get the icecore dataset back 800K years to get data after 0 CE
# to fill in the years not given in the back 2K years file
# located at 
# the site: https://www.ncei.noaa.gov/access/paleo-search/study/6091
# the file: https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc-co2-2008.txt
# reference to the journal article at https://doi.org/10.1038/nature06949
icecore_800K_url = 'https://www.ncei.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc-co2-2008.txt'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

plot_dir = pathlib.Path('./plots')
plot_dir.mkdir(exist_ok=True)

plot_basename = 'co2_2k_ce'

pdf_file = plot_dir / f"{plot_basename}.pdf"
png_file = plot_dir / f"{plot_basename}.png"

## **Load in data and prepare it for plotting**

### Load in MLO record

In [5]:
df_mlo = pd.read_csv(mlo_data_file, sep=',', comment='"', skipinitialspace=True)
df_mlo.head()

,date,co2,date_seas_adj,co2_seas_adj,date_fit,co2_fit,date_seas_adj_fit,co2_seas_adj_fit,date_filled,co2_filled,date_seas_adj_filled,co2_seas_adj_filled
0,1958.202740,315.71,1958.202740,314.43,1958.202740,316.20,1958.202740,314.91,1958.202740,315.71,1958.202740,314.43
1,1958.287671,317.45,1958.287671,315.16,1958.287671,317.30,1958.287671,314.99,1958.287671,317.45,1958.287671,315.16
2,1958.369863,317.51,1958.369863,314.69,1958.369863,317.88,1958.369863,315.07,1958.369863,317.51,1958.369863,314.69
3,1958.536986,315.87,1958.536986,315.20,1958.454795,317.26,1958.454795,315.15,1958.454795,317.26,1958.454795,315.15
4,1958.621918,314.93,1958.621918,316.21,1958.536986,315.86,1958.536986,315.22,1958.536986,315.87,1958.536986,315.20


### Get MLO Seasonally adjusted data

In [6]:
df_mlo = df_mlo[['date_seas_adj', 'co2_seas_adj']].copy()

### Remove rows with fill values NaN

In [7]:
df_mlo = df_mlo.dropna()

### Convert to numpy arrays to use for labeling relevant years in the plot

In [8]:
mlo_date = df_mlo['date_seas_adj'].to_numpy()
mlo_co2 = df_mlo['co2_seas_adj'].to_numpy()


### **Load in the ice-core record back 2K years**

In [9]:
df_icecore_2K = pd.read_csv(icecore_2K_data_file, sep=',', comment='#', skipinitialspace=True)

df_icecore_2K.head()

,Sample ID,Ice Age (year AD),CO2 Age (year AD),CO2 (ppm),Uncert (ppm)
0,DSSW20K 15.8,firn,1996,359.87,0.02
1,DSSW20K 29,firn,1994,357.24,0.04
2,DE08-2 0,firn,1993,354.60,0.02
3,DSSW20K 37.8,firn,1992,353.89,0.04
4,DE08-2 10,firn,1991,352.60,0.18


### Sort ice-core dataframe by CO2 Age

In [10]:
df_icecore_2K = df_icecore_2K.sort_values(by=['CO2 Age (year AD)'], ascending=True)

df_icecore_2K = df_icecore_2K.reset_index(drop=True)

df_icecore_2K.head()

,Sample ID,Ice Age (year AD),CO2 Age (year AD),CO2 (ppm),Uncert (ppm)
0,DSS 135,94,154,278.2,0.4
1,DSS 134,115,175,277.7,0.4
2,DSS 132,181,241,280.4,0.6
3,DSS 131,206,266,278.4,0.6
4,DSS 082,214,274,280.2,2.5


### Get subset of datafreame and rename columns

In [11]:
df_icecore_2K.drop(['Sample ID', 'Ice Age (year AD)', 'Uncert (ppm)'], axis=1, inplace=True)

df_icecore_2K.columns = ['date_ce', 'co2']

df_icecore_2K.head()

,date_ce,co2
0,154,278.2
1,175,277.7
2,241,280.4
3,266,278.4
4,274,280.2


### Filter ice-core data up to 1957 (So as not to overlap with MLO data)

In [12]:
df_icecore_2K = df_icecore_2K[df_icecore_2K['date_ce'] < 1958]

df_icecore_2K.tail()

,date_ce,co2
264,1956,314.40
265,1957,314.50
266,1957,314.40
267,1957,316.45
268,1957,315.10


### Load in the icecore record back 800K years to get data after 0 CE
The date column is in BP units which stands for "before present year"

In [13]:
response = requests.get(icecore_800K_url)
file_text = response.text
text_lines = file_text.split('\n')

#### Use data from section 3 of ice-core file edc-co2-2008.txt labeled: "Composite CO2 record"

**Section to start with**

3. Composite CO2 record (0-800 kyr BP)

**Section to stop with**

end of file

In [14]:
start_section = [i for i in range(len(text_lines))
               if text_lines[i].startswith('3. Composite CO2 record (0-800 kyr BP)')][0]

# Select to end of file since no more sections after Section 3
section_lines = text_lines[start_section:]

# start data selection after line "Age(yrBP)    CO2(ppmv)"
header_end = [i for i in range(len(section_lines))
              if section_lines[i].startswith('Age(yrBP)')][0]

start_data = header_end + 1

data_lines = section_lines[start_data:]

data_lines[1:10]

['268           274.9\r',
 '279           277.9\r',
 '395           279.1\r',
 '404           281.9\r',
 '485           277.7\r',
 '559           281.1\r',
 '672           282.2\r',
 '754           280.1\r',
 '877           278.4\r']

#### Remove trailing return character

In [15]:
data_list = [line.rstrip() for line in data_lines]
data_list[0:10]

['137           280.4',
 '268           274.9',
 '279           277.9',
 '395           279.1',
 '404           281.9',
 '485           277.7',
 '559           281.1',
 '672           282.2',
 '754           280.1',
 '877           278.4']

#### Read data list into a Pandas dataframe and split into columns date_ce and co2

In [16]:
df_icecore_800K = pd.DataFrame(data_list)
df_icecore_800K.columns = ['data']

df_icecore_800K[['date_bp', 'co2']] = df_icecore_800K['data'].str.split(" ", 1, expand=True)
df_icecore_800K.drop('data', axis=1, inplace=True)

df_icecore_800K.head()

,date_bp,co2
0,137,280.4
1,268,274.9
2,279,277.9
3,395,279.1
4,404,281.9


#### Remove empty rows and convert columns from strings to numbers

In [17]:
# First change empty strings to NaN. Then can easily drop NaN rows
df_icecore_800K['date_bp'].replace('', np.nan, inplace=True)

df_icecore_800K = df_icecore_800K.dropna()

df_icecore_800K = df_icecore_800K.astype(float)

### Convert back 800K years dataset from BP dates to CE dates

The date in the 800K dataset is in BP years which means (before present) years. Here in the dataset, present year is 1950

#### First convert 800K dataset from dates BP (before present) to CE

years before present = 1950 - year ce

year ce = 1950 - years before present

In [18]:
df_icecore_800K['date_ce'] = 1950 - df_icecore_800K['date_bp']
df_icecore_800K[0:10]

,date_bp,co2,date_ce
0,137.0,280.4,1813.0
1,268.0,274.9,1682.0
2,279.0,277.9,1671.0
3,395.0,279.1,1555.0
4,404.0,281.9,1546.0
5,485.0,277.7,1465.0
6,559.0,281.1,1391.0
7,672.0,282.2,1278.0
8,754.0,280.1,1196.0
9,877.0,278.4,1073.0


#### Sort date ascending

In [19]:
df_icecore_800K = df_icecore_800K.sort_values(by=['date_ce'], ascending=True)

# And reset the index
df_icecore_800K = df_icecore_800K.reset_index(drop=True)

df_icecore_800K.head()

,date_bp,co2,date_ce
0,798512.0,191.0,-796562.0
1,797099.0,188.4,-795149.0
2,796467.0,189.3,-794517.0
3,795202.0,195.2,-793252.0
4,794608.0,199.4,-792658.0


#### Get data points from the 800K dataset that start at 0 CE and stop before the 2K dataset starts

In [20]:
min_2K = min(df_icecore_2K['date_ce'])

df_icecore_800K_subset = df_icecore_800K[(df_icecore_800K['date_ce'] < min_2K) & (df_icecore_800K['date_ce'] >= 0)]

#### Remove 'date_bp' column to have matching columns when concatenate 2K and 800K datasets

In [21]:
df_icecore_800K_subset = df_icecore_800K_subset.drop('date_bp', axis=1).copy()

# move date_ce column before co2
df_icecore_800K_subset = df_icecore_800K_subset.reindex(columns=['date_ce', 'co2'])

df_icecore_800K_subset.head()


,date_ce,co2
1075,19.0,276.9
1076,138.0,278.0


#### Concatenate the icecore 800K and 2K datasets

In [22]:
df_combined_icecore = pd.concat([df_icecore_800K_subset, df_icecore_2K], ignore_index=True)

df_combined_icecore.head()

,date_ce,co2
0,19.0,276.9
1,138.0,278.0
2,154.0,278.2
3,175.0,277.7
4,241.0,280.4


### Combine MLO and Icecore data

#### Rename columns of MLO dataframe so can align with the icecore dataframe to concatenate

In [23]:
df_mlo.columns = ['date_ce', 'co2']

#### Vertically concatenate the dataframes

In [24]:
df_combined = pd.concat([df_combined_icecore, df_mlo], ignore_index=True)

# And reset the index
df_combined = df_combined.reset_index(drop=True)

df_combined

,date_ce,co2
0,19.000000,276.90
1,138.000000,278.00
2,154.000000,278.20
3,175.000000,277.70
4,241.000000,280.40
...,...,...
1041,2022.789041,418.93
1042,2022.873973,419.31
1043,2022.956164,419.50
1044,2023.041096,419.17


### Convert to numpy arrays for plotting

In [25]:
combined_years = df_combined['date_ce'].to_numpy()
combined_co2 = df_combined['co2'].to_numpy()

## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

### Function to convert a datetime to a float

In [26]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

### Function to convert a float to a datetime

In [27]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)

### Function to create various date versions for today's date

In [28]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

### Function to get the end date of data

In [29]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

<a name="functions-to-modify-graphic"></a>
## **Define functions that can modify the graphic**

### Function to create gradient under curve

In [30]:
# Gradient function modified from the original function in
# the answer from https://stackoverflow.com/questions/29321835/is-it-possible-to-get-color-gradients-under-curve-in-matplotlib

# Modified to add gradient below curve and have a bottom alpha
# The color under the plot line varies as a gradient from the line to the
# bottom axis.

# Modify arguments of the original function to include limits of the 
# gradient under the curve, the transparency and gradient color
# def gradient_fill(x, y, fill_color=None, ax=None, **kwargs):
def gradient_fill(x, y, fill_color='#FFFFFF', xmin=None, ymin=None, xmax=None, 
                  ymax=None, alpha_bottom=None, ax=None, alpha=1, zorder=1, **kwargs):
    """
    Plot a line with a linear alpha gradient filled beneath it.

    Parameters
    ----------
    x, y : array-like
        The data values of the line.
    fill_color : a matplotlib color specifier (string, tuple) or None
        The color for the fill. If None, the color of the line will be used.
    ax : a matplotlib Axes instance
        The axes to plot on. If None, the current pyplot axes will be used.
    Additional arguments are passed on to matplotlib's ``plot`` function.

    Returns
    -------
    line : a Line2D instance
        The line plotted.
    im : an AxesImage instance
        The transparent gradient clipped to just the area beneath the curve.
    """
    if ax is None:
        ax = plt.gca()

    # Only want the gradient to show and not the line defining the gradient shape
    # so change the original function
    #line, = ax.plot(x, y, **kwargs)
    #if fill_color is None:
    #    fill_color = line.get_color()

    if alpha_bottom is None:
        alpha_bottom = 0

    if xmin is None:
        xmin = x.min()

    if ymin is None:
        ymin = y.min()
        
    if xmax is None:
        xmax = x.max()

    if ymax is None:
        ymax = y.max()
        
    if zorder is None:
        zorder = 1
        
    # zorder in alpha of the gradient are now set in the function arguments,
    # so modify this portion of the original function
    #zorder = line.get_zorder()
    
    #alpha = line.get_alpha()
    #alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 1, 4), dtype=float)

    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:, :, :3] = rgb
    #z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]
    z[:, :, -1] = np.linspace(alpha_bottom, alpha, 100)[:, None]

    # Get the gradient limits from the function arguments,
    # so modify the original function
    #xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    xy = np.vstack([[xmin, ymin], xy, [xmax, ymin], [xmin, ymin]])
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)
    
    ax.autoscale(True)
    
    # Modify original function since there is no need 
    # to return the line and im objects
    #return line, im

### Function to set global fonts and linewidth properties

In [31]:
def set_matplotlib_properties():
    
    # Set default properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This causes matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set axes properties of the graphic

In [32]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):

    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)
    
    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=8, width=1)
    
    tick_spacing = 50
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    
    ax.tick_params(which='minor', direction='in', length=4)
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold')

    ax.set_xlabel(xlabel, fontweight='bold', fontsize=12, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    # Set axes limits last 
    # If did before, setting xtick labels past xmin & xmax would have
    # extended the plot limits
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to place the titles

In [33]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [34]:
def add_inset_label(ax, label_text):
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top')


### Function to add arrow annotations

In [35]:
def apply_arrow_annotations(todays_decimal, mlo_date, mlo_co2):
    
    
    #  1960, 1980, 2000, 2020
    # In years ce
    x_1960 = 1960
    x_1980 = 1980
    x_2000 = 2000
    x_2020 = 2020

    spl_fit = interpolate.splrep(mlo_date, mlo_co2)

    spl_val_1960 = interpolate.splev(x_1960, spl_fit)
    spl_val_1980 = interpolate.splev(x_1980, spl_fit)
    spl_val_2000 = interpolate.splev(x_2000, spl_fit)
    spl_val_2020 = interpolate.splev(x_2020, spl_fit)
    
    # Annotation start
    x_start = 1950
    
    plt.annotate('1960',
             xy=[x_start, spl_val_1960],
             xytext=[-60, 0], 
             verticalalignment = "center",
             arrowprops=dict(arrowstyle='->',
                             relpos=(0, 0.5), lw=1),           
             fontsize=11,
             textcoords="offset points")

    plt.annotate('1980',
             xy=[x_start, spl_val_1980],
             xytext=[-60, 0], 
             verticalalignment = "center",
             arrowprops=dict(arrowstyle='->',
                             relpos=(0, 0.5), lw=1),             
             fontsize=11, 
             textcoords="offset points")


    plt.annotate('2000',
             xy=[x_start, spl_val_2000],
             xytext=[-60, 0], 
             verticalalignment = "center",
             arrowprops=dict(arrowstyle='->',
                             relpos=(0, 0.5), lw=1),             
             fontsize=11, 
             textcoords="offset points")


    plt.annotate('2020',
             xy=[x_start, spl_val_2020],
             xytext=[-60, 0], 
             verticalalignment = "center",
             arrowprops=dict(arrowstyle='->',
                             relpos=(0, 0.5), lw=1),           
             fontsize=11, 
             textcoords="offset points")

### Function to save the graphic

In [36]:
def save_graphic(fig, pdf_file, png_file):

    fig_width, fig_height = fig.get_size_inches()

    # For pdf
    width_in = 10
    height_in = (fig_height/fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    # Save to a high dpi so that logo png file has a high resolution
    fig.savefig(pdf_file, facecolor='w', edgecolor='w',
                orientation='landscape', format=None,
                transparent=False, bbox_inches='tight', dpi=600)

    # For png
    # Want a png of height 500px

    # First save figure as a png and then scale to size needed

    png_dpi = 300

    fig.savefig(png_file, facecolor='w', edgecolor='w',
                orientation='landscape', dpi=png_dpi, bbox_inches='tight')

    # Now resize the image to be width 1000px for the
    # keelingcurve website
    img = Image.open(png_file)
    img_width, img_height = img.size

    # width_px = 1000
    # img_scale = (width_px / img_width)
    # height_px = int((float(img_height) * float(img_scale)))

    height_px = 500
    img_scale = (height_px / img_height)
    width_px = int((float(img_width) * float(img_scale)))

    #img = img.resize((width_px, height_px), Image.Resampling.LANCZOS)
    img = img.resize((width_px, height_px), Image.ANTIALIAS)
    img.save(png_file)


### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [37]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))


# **Steps to create the graphic**

### Change any properties in the section [Functions to modify the graphic](#functions-to-modify-graphic)

### Comment out any function in the overall function [create_graphic](#function-to-create-the-graphic) to remove an element from the final graphic

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [38]:
set_matplotlib_properties()

## **Set plot limits and labeling**

#### Get todays date in various formats for labeling|

In [39]:
todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

### Set yaxis min and max limits

In [40]:
ymin = 200
ymax = 430

### Set xaxis min and max limits

In [41]:
xmin = 0
xmax = todays_year + 40

### Set titles and axes labels

In [42]:
xlabel = 'Years (C.E.)'
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

#title1 = get_data_end_date(combined_years)
title1 = ""
title2 = "Mauna Loa data starting in 1958. Ice-core data before 1958."

<a name="function-to-create-the-graphic"></a>
## **Function to create the graphic**

In [43]:
def create_graphic(msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years, combined_co2, btn):

    msg_out.clear_output()
    graphic_out.clear_output()

    # create a figure container
    fig = plt.figure(figsize=(10, 7))

    # Set axes
    # Add padding for exterior plot text
    ax = plt.axes([0.11, 0.12, 0.83, 0.54])

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Graphic') as progress:

        # ------------------------
        # Add gradient under curve
        # ------------------------

        # The gradient is a guide to the eye

        # Set gradient limits to the data limits
        data_start_date = min(combined_years)
        data_end_date = max(combined_years)

        area_color = '#73a7e6'

        gradient_fill(combined_years, combined_co2, fill_color=area_color, 
                    xmin=data_start_date, ymin=ymin, xmax=data_end_date, 
                    ymax=ymax, ax=ax, alpha_bottom=0.1, alpha=1.0)


        # -------------
        # Plot the data
        # -------------

        # Set plotting zorder above the gradient so the plot points are on top
        ax.plot(combined_years, combined_co2, '-', color='black', linewidth=0.75, zorder=5)

        # -------------------
        # Set plot properties
        # -------------------
        set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel)

        # ---------------
        # Add date arrows
        # ---------------
        apply_arrow_annotations(todays_decimal, mlo_date, mlo_co2)

        # --------------
        # Add inset text
        # --------------
        inset_text = f'Last updated {todays_date_modyyr}'
        add_inset_label(ax, inset_text)

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # ------------
        # Save graphic
        # ------------
        fig = plt.gcf()

        save_graphic(fig, pdf_file, png_file)


    with graphic_out:

        plt.show()

## **Button to create and save the graphic**

In [44]:
msg_out = widgets.Output()
graphic_out = widgets.Output()

button = widgets.Button(description='Create Graphic', button_style='primary')
button.on_click(functools.partial(create_graphic, msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, combined_years, combined_co2))
display(button)

display(msg_out)
display(graphic_out)

Button(button_style='primary', description='Create Graphic', style=ButtonStyle())

Output()

Output()

## **Button to download PDF & PNG files of the graphic**

The download button only works when the notebook is run with Google Colab

If the notebook is run on a local machine, the graphics are inside the plots directory

In [45]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(png_file)
        files.download(pdf_file)
    except:
        with msg_out:
            print("If running notebook in Google Colab:")
            print("Need to create the graphic first.")
            print(f"\nIf running in a python environment, the graphics are in the plots directory.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Graphic Files", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Graphic Files',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Graphic Files', layout=Layout(height='auto', width='auto'…

Output()